# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras import losses
import mnist
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np

In [3]:
# your code here
df = pd.read_csv(r'C:\Users\Sebas!\Documents\GitHub\lab-deep-learning\your-code\tic-tac-toe.csv')

df.isnull().sum()

TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

In [4]:
df.dtypes

TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object

In [5]:
df.replace('x', value =1,inplace=True)
df.replace('o', value =2,inplace=True)
df.replace('b', value= 0, inplace=True)

In [6]:
df

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,2,2,1,2,2,True
1,1,1,1,1,2,2,2,1,2,True
2,1,1,1,1,2,2,2,2,1,True
3,1,1,1,1,2,2,2,0,0,True
4,1,1,1,1,2,2,0,2,0,True
...,...,...,...,...,...,...,...,...,...,...
953,2,1,1,1,2,2,2,1,1,False
954,2,1,2,1,1,2,1,2,1,False
955,2,1,2,1,2,1,1,2,1,False
956,2,1,2,2,1,1,1,2,1,False


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [7]:
df.columns

Index(['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'], dtype='object')

In [8]:
df_randomized = df.sample(frac=1).reset_index(drop=True) #This randomize the entire dataset 

In [9]:
# your code here

'''I was practically copying-pasting the documentation https://www.tensorflow.org/tutorials/load_data/pandas_dataframe since we had the same situation, but it gave me a lot of problems when we reach the training step, the main issue was the shape for the features and the layers they MUST be the same due something called scalar values.

My solution to this issue is, since the shape of the features where (9, 2) and the labels where (None,1) I decided to reshape the labels by adding two layers, one for shape and another one applied also to the features due the scalar value'''

features_train = tf.data.Dataset.from_tensor_slices(df_randomized[['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']][:670].values)
label_train = tf.data.Dataset.from_tensor_slices(df_randomized['class'][:670].values).batch(1) #layer 2

features_test = tf.data.Dataset.from_tensor_slices(df_randomized[['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']][670:].values)
label_test = tf.data.Dataset.from_tensor_slices(df_randomized['class'][670:].values).batch(1)

dataset_train_no_scalar = tf.data.Dataset.zip((features_train, label_train))
dataset_test_no_scalar = tf.data.Dataset.zip((features_test, label_test))

In [10]:
for feat, targ in dataset_train_no_scalar.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [0 2 1 0 2 0 1 2 1], Target: [False]
Features: [2 0 2 1 1 1 2 1 0], Target: [ True]
Features: [1 1 2 2 1 1 1 2 2], Target: [False]
Features: [1 0 2 1 2 1 1 0 2], Target: [ True]
Features: [1 1 1 2 2 1 0 0 2], Target: [ True]


In [11]:
train_dataset = dataset_train_no_scalar.batch(1) # extra layer for scalar value
test_dataset = dataset_test_no_scalar.batch(1)

for feat, targ in train_dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [[0 2 1 0 2 0 1 2 1]], Target: [[False]]
Features: [[2 0 2 1 1 1 2 1 0]], Target: [[ True]]
Features: [[1 1 2 2 1 1 1 2 2]], Target: [[False]]
Features: [[1 0 2 1 2 1 1 0 2]], Target: [[ True]]
Features: [[1 1 1 2 2 1 0 0 2]], Target: [[ True]]


In [14]:
def get_compiled_model():  
  model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),

    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), #https://keras.io/api/losses/probabilistic_losses/#sparse_categorical_crossentropy-function
                metrics=['accuracy'])
  return model

model = get_compiled_model()
model.fit(train_dataset, epochs= 5)

Epoch 1/5
670/670 [==============================] - 1s 854us/step - loss: 0.6427 - accuracy: 0.5030
Epoch 2/5
670/670 [==============================] - 1s 883us/step - loss: 0.6427 - accuracy: 0.5134
Epoch 3/5
670/670 [==============================] - 1s 886us/step - loss: 0.6427 - accuracy: 0.5075
Epoch 4/5
670/670 [==============================] - 1s 841us/step - loss: 0.6427 - accuracy: 0.5104
Epoch 5/5
670/670 [==============================] - 1s 841us/step - loss: 0.6427 - accuracy: 0.5149


In [16]:
val_loss, val_acc = model.evaluate(test_dataset)

288/288 [==============================] - 0s 499us/step - loss: 0.6581 - accuracy: 0.0729


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [13]:
# your code here
#model.save('tic_tac_toe.model')

In [14]:
new_model = tf.keras.models.load_model('tic_tac_toe.model')

In [15]:
prediction = new_model.predict([df_randomized[['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']][670:]])

In [16]:
test = df_randomized['class'][670:].values
wrong_predictions = 0
for n in range(0,len(test)):
    predicted_value = bool(np.argmax(prediction[n]))
    if predicted_value != test[n]:
        #print('Prediction:',predicted_value,' Real data:',test[n], ' In row:', n)
        wrong_predictions += 1

print(wrong_predictions,'incorrect predictions out of',len(test),'predictions', wrong_predictions*100/len(test),'%')

108 incorrect predictions out of 288 predictions 37.5 %


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [18]:
# your code here
new_model.add( tf.keras.layers.Dense(128, activation=tf.nn.relu, name='new_layer_1') )
new_model.add(tf.keras.layers.Flatten(name='new_flattener'))
new_model.add( tf.keras.layers.Dense(2, activation=tf.nn.relu, name='new_output_layer') )




In [19]:
new_model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.003, 
                                                      beta_1=0.9, beta_2=0.999, 
                                                      epsilon=1e-07, 
                                                      amsgrad=False),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                metrics=['accuracy'])


In [20]:
new_model.fit(train_dataset, epochs= 5)  

Epoch 1/5
670/670 [==============================] - 1s 953us/step - loss: 0.6525 - accuracy: 0.4239
Epoch 2/5
670/670 [==============================] - 1s 955us/step - loss: 0.6490 - accuracy: 0.7030
Epoch 3/5
670/670 [==============================] - 1s 961us/step - loss: 0.6486 - accuracy: 0.7463
Epoch 4/5
670/670 [==============================] - 1s 1ms/step - loss: 0.6483 - accuracy: 0.7687
Epoch 5/5
670/670 [==============================] - 1s 992us/step - loss: 0.6481 - accuracy: 0.7731


In [20]:
val_loss, val_acc = new_model.evaluate(test_dataset)

288/288 [==============================] - 0s 589us/step - loss: 0.6931 - accuracy: 1.0000


**Which approach(es) did you find helpful to improve your model performance?**

In [21]:
# your answer here
# by trial and error, I suspected that since the learning rate was very low I should be more reserved in increasing it and I believe that I was reaching an overfitting if it was added another layer.